# Preparing datasets

In [1]:
# %config InlineBackend.figure_format='retina'
from ekorpkit import eKonf

eKonf.setLogger("INFO")
print("version:", eKonf.__version__)
print("is notebook?", eKonf.is_notebook())
print("is colab?", eKonf.is_colab())
print("evironment varialbles:")
eKonf.print(eKonf.env().dict())

INFO:ekorpkit.base:IPython version: (6, 9, 0), client: jupyter_client
INFO:ekorpkit.base:Google Colab not detected.


version: 0.1.35+4.ga5901ee.dirty
is notebook? True
is colab? False
evironment varialbles:
{'CUDA_DEVICE_ORDER': None,
 'CUDA_VISIBLE_DEVICES': None,
 'EKORPKIT_CONFIG_DIR': '/workspace/projects/ekorpkit-book/config',
 'EKORPKIT_DATA_DIR': None,
 'EKORPKIT_LOG_LEVEL': 'INFO',
 'EKORPKIT_PROJECT': 'ekorpkit-book',
 'EKORPKIT_WORKSPACE_ROOT': '/workspace',
 'KMP_DUPLICATE_LIB_OK': 'TRUE',
 'NUM_WORKERS': 230}


In [2]:
cfg = eKonf.compose("io/fetcher=labelstudio")
cfg.project_id = 2
ls = eKonf.instantiate(cfg)

INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env
INFO:ekorpkit.base:IPython version: (6, 9, 0), client: jupyter_client
INFO:ekorpkit.io.fetch.base:/workspace/.tmp/fetcher/annotations.json already exists. skipping..


In [37]:
%%time

import pandas as pd

df = pd.read_json(cfg.path.output.filepath)

def to_list(annotations):
    return [[ann['id'], ann['completed_by'], ann['result'][0]['value']['choices'][0]] for ann in annotations if ann['result']]
    
df['annotations'] = df.annotations.apply(to_list)
df['text'] = df.data.apply(lambda x: x['text'])
cols = ['id', 'annotations', 'text']
df = df[cols]
df = df.explode('annotations')
cols = ['annot_id', 'annotator', 'labels']
df[cols] = df.annotations.apply(pd.Series)
df = df.dropna(subset=cols)
cols = ['annot_id', 'annotator']
df[cols] = df[cols].astype(int)
df = df.drop(columns='annotations')
df.head()


CPU times: user 3.81 s, sys: 93.5 ms, total: 3.9 s
Wall time: 3.84 s


,id,text,annot_id,annotator,labels
0,26444,"국내 중소 서버업계 관계자는 '그동안 hpe, 델 등 미국 업체와 경쟁도 힘겨웠는데...",17465,6,Negative
1,26377,"현대차·모비스·lg이노텍 / 자동차 외장램프용 면조명 확산시트\n현대자동차, 현대모...",17449,6,Neutral
2,26426,"현대·기아차, 미국에 신모델 투입\n현대·기아자동차가 다음달부터 미국에 신모델을 잇...",17225,7,Positive
3,26389,"1인승인 ft2는 음성이나 화상 인식 기술을 활용해 운전자의 감정을 읽고, 미리 축...",17222,7,Positive
4,26388,곧 내놓을 신차는 물론 다소 먼 미래에 출시하기 위해 개발 중인 차가 콘셉트카 형태...,17221,7,Neutral


In [43]:
df.groupby('annotator')['annotator'].count()

annotator
1       7
2     340
3    1144
4     868
5    5995
6    3403
7    1956
8    3487
9     183
Name: annotator, dtype: int64

In [47]:
len(df['text'].unique())

13471

In [48]:
len(df['id'].unique())

13606